In [3]:
import sys
import requests
import pandas as pd
import numpy as np
from pprint import pprint

%load_ext autoreload
%autoreload 2
# sys.path.append("D:/Github/note/module")                        # for windows
sys.path.append("/Users/xinc./Documents/GitHub/note/module")    # for mac
from get_info_FinMind import GetInfoFinMind
FinMind_client = GetInfoFinMind()
from get_info_JQC import GetInfoJQC
JQC_client = GetInfoJQC()
from plot_func import plot, plot_scatter, plot_df_columns, plot_pdf, plot_dropped_positions, plot_sequence
from performance_func import summarize_performance, mean_ttest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# import data

In [5]:
df = FinMind_client.get_US_stock("SPY", "2021-01-01", "2021-02-01")
df

2025-11-06 11:25:14.774 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.USStockPrice, data_id: SPY


,date,stock_id,Adj_Close,Close,High,Low,Open,Volume
0,2021-01-04,SPY,345.27,368.79,375.45,364.82,375.31,110210800
1,2021-01-05,SPY,347.65,371.33,372.50,368.05,368.10,66426200
2,2021-01-06,SPY,349.73,373.55,376.98,369.12,369.71,107997700
3,2021-01-07,SPY,354.93,379.10,379.90,375.91,376.10,68766800
4,2021-01-08,SPY,356.95,381.26,381.49,377.10,380.59,71677200
5,2021-01-11,SPY,354.54,378.69,380.58,377.72,377.85,51034700
6,2021-01-12,SPY,354.62,378.77,379.86,376.36,378.89,52547700
7,2021-01-13,SPY,355.57,379.79,380.86,377.85,378.69,45303600
8,2021-01-14,SPY,354.33,378.46,381.13,378.10,380.59,49989100
9,2021-01-15,SPY,351.74,375.70,377.58,373.70,376.72,107160000


In [39]:
fetcher = GetInfoTWSE(certify = False)
base_df = fetcher.init_df(stock_id, start_time, end_time)
base_df = fetcher.get_stock_price()

start = base_df.index[0].strftime("%Y-%m-%d")
end = base_df.index[-1].strftime("%Y-%m-%d")
base_df

c:\Users\ownme\anaconda3\envs\quant\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Open,High,Low,Close,Volume
Timestamp,,,,,
2021-04-12,39.40,39.65,39.00,39.5,65571.0
2021-04-13,39.65,39.85,39.20,39.7,9223.0
2021-04-14,39.30,39.45,39.05,39.2,43501.0
2021-04-15,39.45,39.65,39.15,39.2,14084.0
2021-04-16,39.50,39.50,39.20,39.5,18021.0


In [40]:
fetcher = GetInfoFinMind()
df = fetcher.init_df(stock_id, start, end)
df = fetcher.get_share_holding()
# df = fetcher.get_share_holding()
df

2025-09-14 18:57:44.350 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockShareholding, data_id: 1436


,ForeignInvestmentRemainingShares,ForeignInvestmentShares,ForeignInvestmentRemainRatio,ForeignInvestmentSharesRatio,NumberOfSharesIssued
Timestamp,,,,,
2021-04-12,95308225,325775,99.65,0.34,95634000
2021-04-13,95307225,326775,99.65,0.34,95634000
2021-04-14,95308225,325775,99.65,0.34,95634000
2021-04-15,95311225,322775,99.66,0.33,95634000
2021-04-16,95311225,322775,99.66,0.33,95634000


In [41]:
base_df.index = pd.to_datetime(base_df.index, format = "%Y-%m-%d")
df.index = pd.to_datetime(df.index, format = "%Y-%m-%d")
merge_df = base_df.join(df)
merge_df

,Open,High,Low,Close,Volume,ForeignInvestmentRemainingShares,ForeignInvestmentShares,ForeignInvestmentRemainRatio,ForeignInvestmentSharesRatio,NumberOfSharesIssued
Timestamp,,,,,,,,,,
2021-04-12,39.40,39.65,39.00,39.5,65571.0,95308225,325775,99.65,0.34,95634000
2021-04-13,39.65,39.85,39.20,39.7,9223.0,95307225,326775,99.65,0.34,95634000
2021-04-14,39.30,39.45,39.05,39.2,43501.0,95308225,325775,99.65,0.34,95634000
2021-04-15,39.45,39.65,39.15,39.2,14084.0,95311225,322775,99.66,0.33,95634000
2021-04-16,39.50,39.50,39.20,39.5,18021.0,95311225,322775,99.66,0.33,95634000
